# Neural Style Transfer


## ¿Qué es?

Neural Style Transfer es una técnica de Deep Learning que utiliza 3 imágenes: la imagen de **contenido**, la de **estilo** (por ejemplo, una obra de un artista famoso) y la **imagen de entrada**, que se irá transformando para parecerse a la imagen de contenido pero "pintada" con el estilo de la imagen de estilo.

Por ejemplo, si tomamos como contenido la imagen de esta tortuga y como estilo la pintura de Hokusai *La gran ola de Kanagawa* obtendríamos:

<table>
<tr>
<td>
<img src='https://github.com/tensorflow/models/blob/master/research/nst_blogpost/Green_Sea_Turtle_grazing_seagrass.jpg?raw=1'>
</td>
<td>
<img src='https://github.com/tensorflow/models/blob/master/research/nst_blogpost/The_Great_Wave_off_Kanagawa.jpg?raw=1'>
</td>
</tr>
<tr align="center">
<td colspan="2">
<img src="https://github.com/tensorflow/models/blob/master/research/nst_blogpost/wave_turtle.png?raw=1">
</td>
</tr>
</table>

¿Habría pintado Hokusai algo así?
Esto lo podemos conseguir utilizando la técnica *Neural Style Transfer*, la cual hace uso de las representaciones que hacen internamente de las imágenes las redes neuronales profundas.

##¿Cómo funciona?

Las redes neuronales convolucionales (CNN) son capaces de extraer tanto características simples (bordes, colores, esquinas) como más complejas (figuras, dígitos, formas) de las imágenes permitiendo desarrollar sistemas capaces de "entender" lo que está representado en una imagen.

Utilizando una de estas redes, se definen dos funciones de distancia:
* La primera cuantifica cómo de diferente es el contenido de la imagen que estamos generando respecto del de la *imagen contenido*.
* La segunda cuantifica cómo de diferente es el estilo de la imagen generada vs *imagen de estilo*.
De esta forma, se va construyendo una imagen que minimiza ambas funciones.

La técnica se describe en detalle en [Gatys, L. A., Ecker, A. S., & Bethge, M. (2015). A neural algorithm of artistic style.](https://arxiv.org/abs/1508.06576)

### Seguiremos los siguientes pasos:

1. Visualizar los datos
2. Preparación de los datos
3. Definición de las funciones de pérdida 
4. Creación del modelo
5. Optimización


## Setup

### Descarga de imágenes

In [ ]:
import os
img_dir = '/tmp/nst'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/d/d7/Green_Sea_Turtle_grazing_seagrass.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/0/0a/The_Great_Wave_off_Kanagawa.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/b/b4/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/6/68/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg

### Importar librerías y módulos necesarios

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
mpl.rcParams['axes.grid'] = False

import numpy as np
from PIL import Image
import time
import functools

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf

from tensorflow.python.keras.preprocessing import image as kp_image
from tensorflow.python.keras import models 
from tensorflow.python.keras import losses
from tensorflow.python.keras import layers
from tensorflow.python.keras import backend as K

Utilizaremos [eager execution](https://www.tensorflow.org/guide/eager) de Tensorflow:

In [ ]:
tf.enable_eager_execution()
print("Eager execution: {}".format(tf.executing_eagerly()))

In [ ]:
# Set up some global values here
content_path = '/tmp/nst/Green_Sea_Turtle_grazing_seagrass.jpg'
style_path = '/tmp/nst/The_Great_Wave_off_Kanagawa.jpg'

## Visualización del input

In [ ]:
def load_img(path_to_img):
  max_dim = 512
  img = Image.open(path_to_img)
  long = max(img.size)
  scale = max_dim/long
  img = img.resize((round(img.size[0]*scale), round(img.size[1]*scale)), Image.ANTIALIAS)
  
  img = kp_image.img_to_array(img)
  
  # We need to broadcast the image array such that it has a batch dimension 
  img = np.expand_dims(img, axis=0)
  return img

In [ ]:
def imshow(img, title=None):
  # Remove the batch dimension
  out = np.squeeze(img, axis=0)
  # Normalize for display 
  out = out.astype('uint8')
  plt.imshow(out)
  if title is not None:
    plt.title(title)
  plt.imshow(out)

Visualizamos la imagen de contenido y la imagen de estilo:

In [ ]:
plt.figure(figsize=(10,10))

content = load_img(content_path).astype('uint8')
style = load_img(style_path).astype('uint8')

plt.subplot(1, 2, 1)
imshow(content, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style, 'Style Image')
plt.show()

## Prepación de datos
Para poder cargar y prepropcesar las imágenes de forma rápida, definimos dos funciones para hacer el preprocesado requerido por la red que vamos a utilizar (VGG19): normalizar los inputs y permutar a BGR.



In [ ]:
def load_and_process_img(path_to_img):
  img = load_img(path_to_img)
  img = tf.keras.applications.vgg19.preprocess_input(img)
  return img

Para poder visualizar la imagen generada por la red, necesitamos definir también las anteriores transformaciones pero de forma inversas: 

In [ ]:
def deprocess_img(processed_img):
  x = processed_img.copy()
  if len(x.shape) == 4:
    x = np.squeeze(x, 0)
  assert len(x.shape) == 3, ("Input to deprocess image must be an image of "
                             "dimension [1, height, width, channel] or [height, width, channel]")
  if len(x.shape) != 3:
    raise ValueError("Invalid input to deprocessing image")
  
  # perform the inverse of the preprocessing step
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:,:,::-1]

  x = np.clip(x, 0, 255).astype('uint8')
  return x

### Definir la representación de contenido y estilo
Utilizaremos un modelo preentrenado con la arquitectura VGG19 y tomaremos los resultados de las capas intermedias para realizar el macheo entre la imagen de entrada y las imágenes de estilo y contenido.

Seleccionamos la capa que utilizaremos para la parte de contenido y el conjunto de capas que seleccionaremos para la parte de estilo:


In [ ]:
# Content layer where will pull our feature maps
content_layers = ['block5_conv2'] 

# Style layer we are interested in
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1'
               ]

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

## Construir el modelo 
Tomamos un modelo preentrenado y accedemos al resultado obtenido en las capas intermedias (lo obtendremos tanto para la imagen input como para las de contenido y estilo).

Utilizando Keras [**Functional API**](https://keras.io/getting-started/functional-api-guide/), definir un modelo es tan simple como definir sus inputs y sus outputs: 

`model = Model(inputs, outputs)`

In [ ]:
def get_model():
  """ Creates our model with access to intermediate layers. 
  
  This function will load the VGG19 model and access the intermediate layers. 
  These layers will then be used to create a new model that will take input image
  and return the outputs from these intermediate layers from the VGG model. 
  
  Returns:
    returns a keras model that takes image inputs and outputs the style and 
      content intermediate layers. 
  """
  # Load our model. We load pretrained VGG, trained on imagenet data
  vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  # Get output layers corresponding to style and content layers 
  style_outputs = [vgg.get_layer(name).output for name in style_layers]
  content_outputs = [vgg.get_layer(name).output for name in content_layers]
  model_outputs = style_outputs + content_outputs
  # Build model 
  return models.Model(vgg.input, model_outputs)

## Define and create our loss functions (content and style distances)

Una vez tenemos la representación numérica de cada una de nuestras imágenes en las capas intermedias de la red, debemos definir las funciones de distancia sobre el contenido y sobre el estilo.

### Función de pérdida sobre el contenido

En este caso, la función a minimizar será la distancia euclidea entre la matriz obtenida para la imagen de entrada y la imagen de contenido en la capa *content_outputs*.

Sea $x$ una imagen y $a_{i,j}^l(x)$ el resultado de la red en la capa $l$ sobre la imagen $x$. Sea $C$ la imagen de contenido e $I$ la imagen de entrada, se definirá la función de pérdida como:
$$J_{content}(C,I) = J^l_{content}(C,I) = \sum_{i, j} (a_{i,j}^l(C) - a_{i,j}^l(I))^2$$

Utilizaremos *backpropagation* para minimizar la función anterior.



### Computando la función de pérdida - contenido

In [ ]:
def get_content_loss(base_content, target):
  return tf.reduce_mean(tf.square(base_content - target))

### Función de pérdida sobre estilo

En el caso del estilo, la función a minimizar no es tan directa como en el caso anterior en el que directamente se calcula la distancia entre los outputs intermedios de la imagen de entrada y de la de contenido. En este caso, se comparará la matriz de Gram (matriz de estilo) de los dos outputs, el de la imagen de entrada y el de la imagen de estilo.

Sea $a_{i,j,k}^l(x)$ la activación de $x$ en la capa $l$ siendo $k$ el índice relativo a los $K$ canales de la capa $l$. Se define $G^l$ como una matriz de $K$ x $K$ elementos en la que cada elemento se define como:
$$G^l_{k,k'}=\sum_{i,j}(a^l_{ijk} * a^l_{ijk'})$$

Obtenemos, por tanto, la matriz de Gram en la capa $l$ para la imagen de entrada $G^l(I)$ y para la imagen de estilo $G^l(S)$ y definimos la función de pérdida en la capa $l$ como:
$$J_{style}^l(S,I)=\frac{1}{C}\sum_{k,k'}(G^l_{kk'}(I) - G^l_{kk'}(S))^2$$ 

Para obtener el resultado total sobre todas las capas $l$ contenidas en $L$ calculamos:
$$J_{style}(S,I) = \sum_{l \in L} w_l J_{style}^l(S,I)$$
donde $w_l$ es el peso de cada una de las capas. En nuestro caso, daremos a cada una de ellas el mismo peso ($w_l =\frac{1}{|L|}$)

### Computando la función de pérdida - estilo


In [ ]:
def gram_matrix(input_tensor):
  # We make the image channels first 
  channels = int(input_tensor.shape[-1])
  a = tf.reshape(input_tensor, [-1, channels])
  n = tf.shape(a)[0]
  gram = tf.matmul(a, a, transpose_a=True)
  return gram / tf.cast(n, tf.float32)

def get_style_loss(base_style, gram_target):
  """Expects two images of dimension h, w, c"""
  # height, width, num filters of each layer
  # We scale the loss at a given layer by the size of the feature map and the number of filters
  height, width, channels = base_style.get_shape().as_list()
  gram_style = gram_matrix(base_style)
  
  return tf.reduce_mean(tf.square(gram_style - gram_target))# / (4. * (channels ** 2) * (width * height) ** 2)

## Aplicar *Neural Style Transfer* a nuestras imágenes


La función a minimizar es la que combina tanto el coste del estilo como del contenido:
$$J(I)=\alpha J_{content}(C,I) + \beta J_{style}(S,I)$$

Para ello, se utilizará la técnica del descenso por gradiente y utilizando el algoritmo [Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
En este caso, no se realizará el entrenamiento para actualizar los pesos de la red si no para actualizar los píxeles de la imagen de entrada para minimizar la función de pérdida.


Función auxiliar con el cálculo de los outputs para las imágenes de contenido y estilo:

In [ ]:
def get_feature_representations(model, content_path, style_path):
  """Helper function to compute our content and style feature representations.

  This function will simply load and preprocess both the content and style 
  images from their path. Then it will feed them through the network to obtain
  the outputs of the intermediate layers. 
  
  Arguments:
    model: The model that we are using.
    content_path: The path to the content image.
    style_path: The path to the style image
    
  Returns:
    returns the style features and the content features. 
  """
  # Load our images in 
  content_image = load_and_process_img(content_path)
  style_image = load_and_process_img(style_path)
  
  # batch compute content and style features
  style_outputs = model(style_image)
  content_outputs = model(content_image)
  
  
  # Get the style and content feature representations from our model  
  style_features = [style_layer[0] for style_layer in style_outputs[:num_style_layers]]
  content_features = [content_layer[0] for content_layer in content_outputs[num_style_layers:]]
  return style_features, content_features

### Cálculo del gradiente y la función de pérdida

In [ ]:
def compute_loss(model, loss_weights, init_image, gram_style_features, content_features):
  """This function will compute the loss total loss.
  
  Arguments:
    model: The model that will give us access to the intermediate layers
    loss_weights: The weights of each contribution of each loss function. 
      (style weight, content weight, and total variation weight)
    init_image: Our initial base image. This image is what we are updating with 
      our optimization process. We apply the gradients wrt the loss we are 
      calculating to this image.
    gram_style_features: Precomputed gram matrices corresponding to the 
      defined style layers of interest.
    content_features: Precomputed outputs from defined content layers of 
      interest.
      
  Returns:
    returns the total loss, style loss, content loss, and total variational loss
  """
  style_weight, content_weight = loss_weights
  
  # Feed our init image through our model. This will give us the content and 
  # style representations at our desired layers. Since we're using eager
  # our model is callable just like any other function!
  model_outputs = model(init_image)
  
  style_output_features = model_outputs[:num_style_layers]
  content_output_features = model_outputs[num_style_layers:]
  
  style_score = 0
  content_score = 0

  # Accumulate style losses from all layers
  # Here, we equally weight each contribution of each loss layer
  weight_per_style_layer = 1.0 / float(num_style_layers)
  for target_style, comb_style in zip(gram_style_features, style_output_features):
    style_score += weight_per_style_layer * get_style_loss(comb_style[0], target_style)
    
  # Accumulate content losses from all layers 
  weight_per_content_layer = 1.0 / float(num_content_layers)
  for target_content, comb_content in zip(content_features, content_output_features):
    content_score += weight_per_content_layer* get_content_loss(comb_content[0], target_content)
  
  style_score *= style_weight
  content_score *= content_weight

  # Get total loss
  loss = style_score + content_score 
  return loss, style_score, content_score

In [ ]:
def compute_grads(cfg):
  with tf.GradientTape() as tape: 
    all_loss = compute_loss(**cfg)
  # Compute gradients wrt input image
  total_loss = all_loss[0]
  return tape.gradient(total_loss, cfg['init_image']), all_loss

### Optimización

In [ ]:
import IPython.display

def run_style_transfer(content_path, 
                       style_path,
                       num_iterations=1000,
                       content_weight=1e3, 
                       style_weight=1e-2): 
  # We don't need to (or want to) train any layers of our model, so we set their
  # trainable to false. 
  model = get_model() 
  for layer in model.layers:
    layer.trainable = False
  
  # Get the style and content feature representations (from our specified intermediate layers) 
  style_features, content_features = get_feature_representations(model, content_path, style_path)
  gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]
  
  # Set initial image
  init_image = load_and_process_img(content_path)
  init_image = tf.Variable(init_image, dtype=tf.float32)
  # Create our optimizer
  opt = tf.train.AdamOptimizer(learning_rate=5, beta1=0.99, epsilon=1e-1)

  # For displaying intermediate images 
  iter_count = 1
  
  # Store our best result
  best_loss, best_img = float('inf'), None
  
  # Create a nice config 
  loss_weights = (style_weight, content_weight)
  cfg = {
      'model': model,
      'loss_weights': loss_weights,
      'init_image': init_image,
      'gram_style_features': gram_style_features,
      'content_features': content_features
  }
    
  # For displaying
  num_rows = 2
  num_cols = 5
  display_interval = num_iterations/(num_rows*num_cols)
  start_time = time.time()
  global_start = time.time()
  
  norm_means = np.array([103.939, 116.779, 123.68])
  min_vals = -norm_means
  max_vals = 255 - norm_means   
  
  imgs = []
  for i in range(num_iterations):
    grads, all_loss = compute_grads(cfg)
    loss, style_score, content_score = all_loss
    opt.apply_gradients([(grads, init_image)])
    clipped = tf.clip_by_value(init_image, min_vals, max_vals)
    init_image.assign(clipped)
    end_time = time.time() 
    
    if loss < best_loss:
      # Update best loss and best image from total loss. 
      best_loss = loss
      best_img = deprocess_img(init_image.numpy())

    if i % display_interval== 0:
      start_time = time.time()
      
      # Use the .numpy() method to get the concrete numpy array
      plot_img = init_image.numpy()
      plot_img = deprocess_img(plot_img)
      imgs.append(plot_img)
      IPython.display.clear_output(wait=True)
      IPython.display.display_png(Image.fromarray(plot_img))
      print('Iteration: {}'.format(i))        
      print('Total loss: {:.4e}, ' 
            'style loss: {:.4e}, '
            'content loss: {:.4e}, '
            'time: {:.4f}s'.format(loss, style_score, content_score, time.time() - start_time))
  print('Total time: {:.4f}s'.format(time.time() - global_start))
  IPython.display.clear_output(wait=True)
  plt.figure(figsize=(14,4))
  for i,img in enumerate(imgs):
      plt.subplot(num_rows,num_cols,i+1)
      plt.imshow(img)
      plt.xticks([])
      plt.yticks([])
      
  return best_img, best_loss 

In [ ]:
best, best_loss = run_style_transfer(content_path, 
                                     style_path, num_iterations=1000)

In [ ]:
Image.fromarray(best)

Descarga de la imagen:

In [ ]:
from google.colab import files
Image.fromarray(best).save('wave_turtle.png')
files.download('wave_turtle.png')

## Visualización de resultados

In [ ]:
def show_results(best_img, content_path, style_path, show_large_final=True):
  plt.figure(figsize=(10, 5))
  content = load_img(content_path) 
  style = load_img(style_path)

  plt.subplot(1, 2, 1)
  imshow(content, 'Content Image')

  plt.subplot(1, 2, 2)
  imshow(style, 'Style Image')

  if show_large_final: 
    plt.figure(figsize=(10, 10))

    plt.imshow(best_img)
    plt.title('Output Image')
    plt.show()

In [ ]:
show_results(best, content_path, style_path)

## Prueba con otras imágenes

### Noche estrellada (Van Gogh) + Tuebingen

In [ ]:
show_results(None, 
             '/tmp/nst/Tuebingen_Neckarfront.jpg',
             '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg',
             False)

In [ ]:
best_starry_night, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg',
                                                  '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

In [ ]:
show_results(best_starry_night, '/tmp/nst/Tuebingen_Neckarfront.jpg',
             '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

### Pilares de la creación (NASA) + Tuebingen

In [ ]:
show_results(None, 
             '/tmp/nst/Tuebingen_Neckarfront.jpg',
             '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg',
             False)

In [ ]:
best_poc_tubingen, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg', 
                                                  '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg')

In [ ]:
show_results(best_poc_tubingen, 
             '/tmp/nst/Tuebingen_Neckarfront.jpg',
             '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg')

### Composición VII (Kandinsky) + Tuebingen

In [ ]:
show_results(None, 
             '/tmp/nst/Tuebingen_Neckarfront.jpg', 
             '/tmp/nst/Vassily_Kandinsky,_1913_-_Composition_7.jpg',
             False)

In [ ]:
best_kandinsky_tubingen, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg', 
                                                  '/tmp/nst/Vassily_Kandinsky,_1913_-_Composition_7.jpg')

In [ ]:
show_results(best_kandinsky_tubingen, 
             '/tmp/nst/Tuebingen_Neckarfront.jpg',
             '/tmp/nst/Vassily_Kandinsky,_1913_-_Composition_7.jpg')

### Haz tu propia combinación!
1. Sube una foto tuya a la ventana de archivos (símbolo de la carpeta a la izquierda) junto con un cuadro con el que quieras combinarla.

2. Modifica *foto_sara.jpg* por el nombre de tu foto.
 
3. Modifica *picasso.jpg* por el nombre de la pintura que hayas elegido.

4. Ejecuta las 3 celdas y tendrás tu resultado descargado en tu ordenador!!




In [ ]:
show_results(None, 
             'foto_sara.jpg',
             'picasso.jpg',
             False)

In [ ]:
best_sara_picasso, best_loss = run_style_transfer('foto_sara.jpg', 
                                                  'picasso.jpg')

In [ ]:
show_results(best_sara_picasso, 
             'foto_sara.jpg',
             'picasso.jpg')

In [ ]:
#from google.colab import files
Image.fromarray(best_sara_picasso).save('tu_obra.png')
files.download('tu_obra.png')

Notebook/Texto: https://medium.com/tensorflow/neural-style-transfer-creating-art-with-deep-learning-using-tf-keras-and-eager-execution-7d541ac31398

**[Image of Tuebingen](https://commons.wikimedia.org/wiki/File:Tuebingen_Neckarfront.jpg)** 
Photo: Andreas Praefcke, CC BY 3.0 <https://creativecommons.org/licenses/by/3.0>, via Wikimedia Commons

**[Image of Green Sea Turtle](https://commons.wikimedia.org/wiki/File:Green_Sea_Turtle_grazing_seagrass.jpg)**
P.Lindgren, CC BY-SA 3.0 <https://creativecommons.org/licenses/by-sa/3.0>, via Wikimedia Commons

